In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

import scanpy as sc
import scvi

In [2]:
dataset1 = sc.read_h5ad("GSE124395_labeled.h5ad")
dataset2 = sc.read_h5ad("gse158723_labeled.h5ad")
dataset3 = sc.read_h5ad("healthy_cirrhotic_human_liver_labeled.h5ad")
dataset4 = sc.read_h5ad("liver_immune_cells_human_liver_labeled.h5ad")
dataset5 = sc.read_h5ad("sc_landscape_human_liver_labeled.h5ad")

dataset1.obs_names_make_unique()
dataset2.obs_names_make_unique()
dataset3.obs_names_make_unique()
dataset4.obs_names_make_unique()
dataset5.obs_names_make_unique()

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


In [3]:
# Set up reference data
# concatenate datasets
adata = dataset1.concatenate(dataset3, dataset4, dataset5)
adata.layers["counts"] = adata.X.copy()
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.raw = adata  # keep full dimension safe
sc.pp.highly_variable_genes(
    adata,
    n_top_genes=2000,
    layer="counts",
    batch_key="batch",
    subset=True
)

... storing 'leiden' as categorical
... storing 'celltype' as categorical
... storing 'cell_names' as categorical
... storing 'input_id' as categorical
/home/kevin/anaconda3/envs/scvi/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [4]:
scvi.data.setup_anndata(adata, 
                        layer="counts",
                        batch_key="batch",
                        labels_key="celltype")

INFO     Using batches from adata.obs["batch"]                                               
INFO     Using labels from adata.obs["celltype"]                                             
INFO     Using data from adata.layers["counts"]                                              
INFO     Computing library size prior per batch                                              
INFO     Successfully registered anndata object containing 131899 cells, 2000 vars, 4        
         batches, 11 labels, and 0 proteins. Also registered 0 extra categorical covariates  
         and 0 extra continuous covariates.                                                  
INFO     Please do not further modify adata until model is trained.                          


In [5]:
vae = scvi.model.SCVI(adata, n_layers=2, n_latent=30)

# vae = scvi.model.SCVI.load("liver_scVI_model", adata)

In [6]:
vae.train(use_gpu=True)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 61/61: 100%|██████████| 61/61 [12:05<00:00, 11.89s/it, loss=357, v_num=1]


In [7]:
vae.save("vae_scarches_trained", overwrite=True)

In [19]:
dataset2 = dataset2[:, list(dataset2.var_names)]

In [20]:
test = set(dataset2.var_names).intersection(set(adata.var_names))

In [21]:
len(test)

1693

In [17]:
adata.var_names

Index(['A2M', 'AARD', 'ABCA13', 'ABCA9', 'ABCC2', 'ABCC9', 'ABI3BP', 'ABL2',
       'AC002044.3', 'AC002306.1',
       ...
       'ZNF528-AS1', 'ZNF556', 'ZNF578', 'ZNF683', 'ZNF765', 'ZNF778',
       'ZNF781', 'ZSCAN23', 'ZSWIM4', 'ZSWIM6'],
      dtype='object', length=2000)

In [22]:
vae_q = scvi.model.SCVI.load_query_data(
    dataset2,
    vae,
    inplace_subset_query_vars=True
)

KeyError: "Values ['AC002044.3', 'AC002306.1', 'AC002551.1', 'AC003973.3', 'AC004552.1', 'AC005480.1', 'AC007906.1', 'AC007950.2', 'AC008914.1', 'AC009303.2', 'AC009495.3', 'AC009948.5', 'AC009950.1', 'AC018742.1', 'AC020931.1', 'AC021106.1', 'AC021224.1', 'AC022182.3', 'AC058791.1', 'AC078883.3', 'AC090617.1', 'AC092069.1', 'AC100830.3', 'AC104623.2', 'AC123023.1', 'AC133644.2', 'AC135050.5', 'ADCYAP1', 'ADPGK-AS1', 'AHSP', 'AL022326.1', 'AL109767.1', 'AL590235.1', 'ALAS2', 'ANKRD34B', 'AP000350.5', 'AP000442.1', 'AP4B1-AS1', 'APCDD1L-AS1', 'ARMCX5-GPRASP2', 'ASB14', 'ASH1L-IT1', 'BCYRN1', 'BIRC6-AS2', 'C10orf10', 'C4orf26', 'C8orf4', 'C8orf49', 'C9orf47', 'CA1', 'CACNA1E', 'CCDC144NL-AS1', 'CCL3L3', 'CDH20', 'CEBPB-AS1', 'CEND1', 'CERKL', 'CPB2-AS1', 'CRYM-AS1', 'CTGF', 'CXCR1', 'CYP4F35P', 'DBH-AS1', 'DGKK', 'DISC1FP1', 'DNM3OS', 'DOCK9-AS1', 'DUBR', 'DUXAP8', 'EAF1-AS1', 'EDRF1-AS1', 'EFCAB8', 'EP300-AS1', 'EPB42', 'EXTL3-AS1', 'FAM182B', 'FAM196B', 'FAM198B', 'FAM26F', 'FAM53B-AS1